# First clas Functions
i.e. A function can be used as an object.

**Higher order Function**: if a function accepts a function as argument or returns a function, it's called a higher order function. Eg: `map()`

**Free Variable**: 
```python
>>> def bar():
...   x = 1
...   def foo():
...     print(x)
...     print(locals())
...   foo()
... 
>>> bar()
1
{'x':1}
```
In code above, x is a free variable cuz it's not defined within the inner function, but we still have access to it in the inner function

A free variable is not defined in the current environment, (i.e. collection of local variables), and is also not a global variable! Therefore it must be defined elsewhere. And this is the concept of closures. In Code above, `foo()` closes on x defined in `bar()`.- [Source](https://stackoverflow.com/a/12919351/2365231)

### Videos
* [First Class Function](https://www.youtube.com/watch?v=kr0mpwqttM0)
* [Closures](https://www.youtube.com/watch?v=swU3c34d2NQ)
* [Decorators](https://youtu.be/FsAPt_9Bf3U)

In [1]:
# normal
def square(x):
    return x*x

f = square(5)

print(square)
print(f)

<function square at 0x0000021AD4506BF8>
25


In [3]:
# assign a function to a variable
def square(x):
    return x*x

f = square

print(square)
print(f)
print(f(5))

<function square at 0x0000021AD4506730>
<function square at 0x0000021AD4506730>
25


In [4]:
def square(x):
    return x*x

# custom built map function
def my_map(func, arg_list):
    result = []
    for i in arg_list:
        result.append(func(i))
    return result

squares = my_map(square, [1,2,3,4,5])
print(squares)

[1, 4, 9, 16, 25]


# Closures

In [25]:
def outer_func():
    
    msg = 'Hi'
    # msg is a free variable
    
    def inner_func():
        print(msg)
    
    return inner_func() # with executing

outer_func()
my_func = outer_func()
print(my_func)

Hi
Hi
None


In [34]:
def outer_func():
    
    msg = 'Hi'
    # msg is a free variable
    
    def inner_func():
        print(msg)
    
    return inner_func # without executing

my_func = outer_func()
print(my_func) # gives memory address of returned func
print(my_func.__name__) # name of returned func
print(outer_func().__name__) # name of returned func

my_func() # execute returned func

<function outer_func.<locals>.inner_func at 0x0000021AD496E6A8>
inner_func
inner_func
Hi


In [35]:
def outer_func(msg):
    
    def inner_func():
        print(msg)
    
    return inner_func # without executing

hi_func = outer_func('Hi')
hello_func = outer_func('Hello')

hi_func()
hello_func()

Hi
Hello


In [41]:
# when the inner func also takes parameters
def html_tag(tag):

    def wrap_text(msg):
        print('<{0}>{1}</{0}>'.format(tag, msg))

    return wrap_text

print_h1 = html_tag('h1')
print_h1('Test Headline!')
print_h1('Another Headline!')

print_p = html_tag('p')
print_p('Test Paragraph!')

<h1>Test Headline!</h1>
<h1>Another Headline!</h1>
<p>Test Paragraph!</p>


In [40]:
def logger(msg):
    
    # msg is a free variable 
    def log_message():
        print('Log:',msg)
    
    return log_message

# log_hi = logger('Hi')
# log_hi()

logger('Hi')()

Log: Hi


In [45]:
# Example of a decorator
import logging
logging.basicConfig(filename='closures.log', level=logging.INFO)


def logger(func):
    # *args allows for passing arbitrary number of arguments to func
    def log_func(*args):
        logging.info(
            'Running "{}" with arguments {}'.format(func.__name__, args))
        print(func(*args))
    return log_func


def add(x, y):
    return x+y


def sub(x, y):
    return x-y

add_logger = logger(add)
sub_logger = logger(sub)

add_logger(3, 3)
add_logger(4, 5)

sub_logger(10, 5)
sub_logger(20, 10)

6
9
5
10


# Decorators
```python
@decorator_function
def display():
    print('display function ran')
display()
```
is same as
```python
def display():
    print('display function ran') 
display = decorator_function(display)
display()
```

In [57]:
def decorator_function(original_function):
    def wrapper_function():
        print(f'Wrapper executed this before "{original_function.__name__}"')
        return original_function()
    return wrapper_function

@decorator_function
def display():
    print('display function ran')
    
# display = decorator_function(display)

display()

Wrapper executed this before "display"
display function ran


## `*args`, `**kwargs`
Create a deocrator function which can take misc number of arguments `*args`, `**kwargs`

In [65]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print(f'Wrapper executed this before "{original_function.__name__}"')
        return original_function(*args, **kwargs)
    return wrapper_function

@decorator_function
def display():
    print('display function ran')
    
@decorator_function
def display_info(name, age):
    print(f'Display info ran with arguments ({name}, {age})')

display_info('John', 25)
display()

Wrapper executed this before "display_info"
Display info ran with arguments (John, 25)
Wrapper executed this before "display"
display function ran


# Class decorators
Converting below `decorator function` to `decorator class`
```python
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print(f'Wrapper executed this before "{original_function.__name__}"')
        return original_function(*args, **kwargs)
    return wrapper_function
```

In [68]:
class decorator_class():
    def __init__(self, original_function):
        self.original_function = original_function
    
    def __call__(self, *args, **kwargs):
        print(f'Call method executed this before "{self.original_function.__name__}"')
        return self.original_function(*args, **kwargs)

@decorator_class
def display():
    print('display function ran')
    
@decorator_class
def display_info(name, age):
    print(f'Display info ran with arguments ({name}, {age})')

display_info('John', 25)
display()

Call method executed this before "display_info"
Display info ran with arguments (John, 25)
Call method executed this before "display"
display function ran


## Practical example for decorators

In [2]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    def wrapper(*args, **kwargs):
        logging.info('Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)

    return wrapper

@my_logger
def display_info(name, age):
    print(f'Display info ran with arguments ({name}, {age})')

display_info('John', 25)

Display info ran with arguments (John, 25)


In [6]:
def my_timer(orig_func):
    import time

    # @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

import time

@my_timer
def display_info(name, age):
    time.sleep(1)
    print(f'Display info ran with arguments ({name}, {age})')

display_info('John', 25)

Display info ran with arguments (John, 25)
display_info ran in: 1.0009632110595703 sec


## Chaining Decorators

```python
@my_logger
@my_timer
def display_info:
    pass
```
is same as 
```python
display_info = my_logger(my_timer(display_info))
```

Therefore, it's a good idea to preserve the information of our original function when we use decorators

In [7]:
from functools import wraps


def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)

    return wrapper


def my_timer(orig_func):
    import time

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

import time


@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Tom', 22)

display_info ran with arguments (Tom, 22)
display_info ran in: 1.0012564659118652 sec
